# Clustering and PCA

### Mushroom Dataset

Podeis obtener el conjunto de datos en el siguiente enlace:

[Mushroom Dataset](https://www.kaggle.com/uciml/mushroom-classification)

Como podréis comprobar, hay muchas variables, todas ellas categóricas, por lo que exploraciones con scatterplot no nos serán útiles como en otros casos.

La variable a predecir ``class`` es binaria.


In [25]:
# Carga de librerías, las que hemos considerado básicas, añadid lo que queráis :)

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [26]:
#Ruta para guardar las tablas y reportes
ruta='C:/4_F5/012_noSupervisado/Nat_noSupervisado/'

In [27]:
import pandas as pd
import os

def load_data(file_path):
    """
    Carga los datos del archivo CSV.

    Args:
        file_path (str): Ruta al archivo CSV.

    Returns:
        pd.DataFrame: DataFrame con los datos cargados o None si hay un error.
    """

    # Obtener la ruta absoluta del archivo para evitar problemas con rutas relativas
    abs_file_path = os.path.abspath(file_path)

    # Verificar si el archivo existe en la ruta especificada
    if not os.path.exists(abs_file_path):
        # Imprimir mensajes de error detallados si el archivo no existe
        print(f"Error: El archivo no existe en la ruta: {abs_file_path}")
        print(f"Directorio actual: {os.getcwd()}")  # Mostrar el directorio actual para ayudar a depurar
        print("Contenido del directorio:")
        try:
            # Intentar listar el contenido del directorio para ayudar a identificar el problema
            print(os.listdir(os.path.dirname(abs_file_path)))
        except FileNotFoundError:
            print("El directorio no existe.")  # Informar si el directorio tampoco existe
        return None  # Devolver None para indicar que la carga falló

    try:
        # Intentar cargar los datos utilizando pandas
        df = pd.read_csv(abs_file_path)
        print(f"Datos cargados exitosamente. Shape: {df.shape}")  # Confirmar la carga exitosa
        return df
    except Exception as e:
        # Capturar cualquier excepción que ocurra durante la carga e imprimir un mensaje de error
        print(f"Error al cargar los datos: {e}")
        return None



In [ ]:
# Bloque de ejemplo de uso (solo se ejecuta si el script se ejecuta directamente)
if __name__ == "__main__":
    # Obtener la ruta al archivo CSV dentro de la estructura del proyecto
    #script_dir = os.path.dirname(os.path.abspath(__file__)) #sólo sirve cuando es .py porque la variable __file__ no está disponible porque no se está ejecutando un script de Python directamente.
    #project_root = os.path.dirname(os.path.dirname(script_dir))
    #data_path = os.path.join(project_root, "data", "mushrooms.csv") #(project_root,"carpeta","subcarpeta","archivo.csv")
    data_path = data_path = os.path.join(os.getcwd(), "data", "mushrooms.csv")

    print(f"Intentando cargar el archivo desde: {data_path}")


    # Llamar a la función load_data para cargar los datos
    
    df = load_data(data_path)
    if df is not None:
        # Imprimir las primeras filas del DataFrame si la carga fue exitosa
        print(df.head())
    else:
        # Imprimir un mensaje de error si la carga falló
        print("No se pudieron cargar los datos. Verifica la ruta del archivo y su existencia.")

### Leer conjunto de datos y primer vistazo

In [ ]:
# Leer el csv y sacar por pantalla las cinco primeras filas.

def lectura_csv(df):
    return df.head()
print(lectura_csv(df))

La librería OS nos permite navegar entre los directorios del sistema

### Exploración de datos

In [ ]:
# Descripción del conjunto de datos, estándard.


print(df.describe(include='object').T) # con el parámetro include='object' para obtener estadísticas descriptivas para las columnas categóricas.

In [ ]:
# Información sobre el tipo de datos de cada feature.
df.info()

In [ ]:
print(f'El número de filas del dataset es:\n{df.shape[0]:,}')

In [ ]:

# Igual que otras veces, una linea, contar los nulos por variable.
def count_missing(df):
    missing_values = df.isnull().sum()
    if (missing_values == 0).all():
        return "El conteo por columna de valores 'missing' resulta 0. Es decir, no hay valores 'missing'"
    else:
        return missing_values

print(count_missing(df))

In [ ]:
#---- Verifica si hay columnas duplicadas
duplicated_cols = df.columns[df.columns.duplicated()]

if len(duplicated_cols) > 0:
    print("Columnas duplicadas:", duplicated_cols)
else:
    print("No hay columnas duplicadas")

In [ ]:
for col in df.columns:
    print(df[col].value_counts())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Crear una figura con múltiples subplots
fig, axs = plt.subplots(nrows=23, ncols=1, figsize=(8, 60))

# Iterar sobre cada variable y crear un gráfico de barras
for i, variable in enumerate(df.columns):
    sns.countplot(x=variable, data=df, ax=axs[i])
    axs[i].set_title(f'Distribución de {variable}')

# Ajustar el layout para que los títulos no se superpongan
plt.tight_layout()

# Mostrar el gráfico
plt.show()

In [ ]:
#Considero que valores 'outliers' son aquellos que suponen un 1% del total de registros
for col in df.columns:
    print(df[col].value_counts())

#### Buscar valores extraños. Para ello, ver los valores únicos en cada feature

In [ ]:
# Obtener un nuevo dataframe de dos columnas donde en la primera estén las features (features) y en la otra los valores únicos
# asociados (n_values).

values_unique=pd.DataFrame({'categories':df.nunique()})
print(values_unique)

#### Tratar aquellos valores que entendamos que sean nulos


In [39]:
# Imputaciones. Podéis quitar esos puntos (fila entera), imputar con la moda o dejar ese valor como una posibilidad más.

#### Mirad cuántos valores hay en cada feature, ¿Todas las features aportan información? Si alguna no aporta información, eliminadla

In [ ]:
# Dejar por el camino si procede.
#Diluyo el df para que se expanda cada features según su categoría
melted_df = pd.melt(df, var_name='Features', value_name='Categories')
grouped_df = melted_df.groupby(['Features', 'Categories']).size().reset_index(name='count_values')

category_totals = grouped_df.groupby('Features')['count_values'].transform('sum')
#machaco la variable
grouped_df['ratio']=round((grouped_df['count_values'] / category_totals ),2)
print(grouped_df)

In [41]:
grouped_df.to_csv(ruta+'tables/df_grouped.csv')

In [ ]:
#Voy a filtrar el dataframe agrupado para que --> ¿Qué hago con los ?
#count = (grouped_df['ratio'] < 0.01).sum()
under_one = grouped_df[grouped_df['ratio'] < 0.01]
print(f"Valores menores a 0.01 en la columna 'ratio': {(grouped_df['ratio'] < 0.01).sum()}")
print(under_one)

#### Separar entre variables predictoras y variables a predecir

In [ ]:
# La variable que trata de predecir este conjunto de datos es 'class'.
# Por tanto la variable objetivo es 'y'
# las variables predictoras son las columnas son 'x'
y = df['class']
print(y.head())
x =df.drop(columns='class')

print(x.head())

In [44]:
########## Hago ingeniería de características?

In [ ]:
##### Miro la correlación entre las variables predictoras para ello hay que codificar primero, pero es mejor para variables cuantitativas.


#### Codificar correctamente las variables categóricas a numéricas

----> Creo que es error

 One Hot Encoder necesita ver toda la información de las variables categóricas en X para codificarlas correctamente. Si aplicas el One Hot Encoder después de dividir X en X_train y X_test, podrías perder información importante y obtener resultados inconsistentes.

In [ ]:
# One Hot Encoder (una linea).


#### Train test split

----> la codificación de etiquetas debe realizarse después de la partición de los datos en conjuntos de train y test, ya que de lo contrario se podría producir un problema de "fuga de datos" (data leakage), donde el modelo se beneficia de información que no debería tener acceso.

In [ ]:
from sklearn.model_selection import train_test_split
# Os lo dejamos a todos igual
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

print(x_train)

In [ ]:
# ---- Me resulta demasiadas columnas, voy a ver si con Label. Label no, porque aporta 'ordinalidad'
""" 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x_train_lab=le.fit_transform(x_train.values)
x_test_lab=le.transform(x_test.values) # se garantiza que el modelo no tenga acceso a información que no debería tener y que las categorías se asignen de manera consistente en ambos conjuntos de datos.

"""


De la siguiente manera, puedes asegurarte de que la codificación one-hot se aplique de manera consistente a ambos conjuntos de datos, y que el modelo de codificación se ajuste solo a los datos de entrenamiento.

Hay que recordar que, si no separamos los conjuntos de datos antes de la codificación, podríamos estar introduciendo sesgo en el modelo, ya que el conjunto de prueba podría influir en la codificación one-hot.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Aplicar One Hot Encoder a X
ohe = OneHotEncoder(handle_unknown='ignore') #
ohe_ajuste=ohe.fit_transform(x_train) # se ajusta el modelo de codificación para que sepa cómo codificar las categorías presentes en el conjunto de entrenamiento (train)
x_train_encoded =ohe.transform(x_train) #se aplica la codificación ajustada en fit al conjunto de entrenamiento (train)

x_test_encoded= ohe.transform(x_test) #se aplica la codificación ajustada en fit al conjunto de test 


# Convierte la matriz de salida en un dataframe
df_x_train_enc = pd.DataFrame(x_train_encoded.toarray(), columns=ohe.get_feature_names_out())
df_x_test_enc = pd.DataFrame(x_test_encoded.toarray(), columns=ohe.get_feature_names_out())

print(df_x_train_enc)
print(df_x_test_enc)

## PCA

In [ ]:
#La matriz de correlación puede no ser la mejor opción para analizar la relación entre variables categóricas. En su lugar, podrías utilizar métricas de asociación como la chi-cuadrado o la métrica de información mutua para analizar la relación entre variables categóricas.

In [ ]:
#La reducción de dimensionalidad mediante PCA asume una distrución normal de los datos.

Es un conjunto de datos del que aún no hemos visto nada (no tenemos graficas) así que vamos a hacer algunas. Tenemos el problema de que son muchas variables, **PCA al rescate**: le pedimos que nos de dos dimensiones y las pintamos, sabemos que serán **aquellas que retengan más información**.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.90)      # metodo de sklearn
pca.fit(x_train_encoded) # método fit solo ajusta los parámetros del modelo PCA según los datos de entrenamiento


# Aplicar la transformación PCA a ambos conjuntos de datos
x_train_pca = pca.transform(x_train_encoded)
x_test_pca = pca.transform(x_test_encoded)
pd.DataFrame(x_test_encoded.toarray(), columns=ohe.get_feature_names_out())

df_x_train_pca=pd.DataFrame(x_train_pca.toarray(), columns=pca.get_feature_names_out())
print(df_x_train_pca)
# Representar en un scatterplot y poner en color las etiquetas de entrenamiento

Parece que está bastante separadito, parece que a ojo mucho se puede ver :)

Igualmente, vamos a entrenar un clasificador a ver qué tal lo hace antes de editar más

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. Definir el clasificador y el número de estimadores
# 2. Entrenar en train
# 3. Calcular la precisión sobre test

Es un conjunto sencillo y Random Forest es muy bueno en su trabajo, Igualmente, vamos a ver qué tamaño tenemos de dataset:


In [ ]:
X_train.shape

¿Muchas features no? Vamos a reducir las usando PCA.

In [ ]:
n_features = # definir un rango de valores a probar
scores = []

for n in n_features:

    # Hacer PCA sobre X_train
    # 1. Definir PCA
    # 2. Aprender PCA sobre X_train

    # Entrenar Random Forest
    # 1. Definir el RF
    # 2. Entrenar clasificador

    # Guardar el score


sns.lineplot(x=n_features, y=scores)


Vale, estamos viendo que a partir de unas 10 features ya tenemos el score que queríamos y además hemos reducido las variables a un 10% de las que teníamos, incluso menos que las variables originales.

## Clustering

Viendo que el conjunto de datos es sencillito, podemos intentar hacer algo de clustering a ver qué información podemos obtener.

El primer paso va a ser importar la función de Kmeans de sklearn, y a partir de ahi, vamos a buscar el valor óptimo de clusters. Como hemos visto anteriormente, este valor lo obtenemos, por ejemplo, del codo de la gráfica que representa el total de las distancias de los puntos a los centros de los clusters asociados. Os dejo la página de la documentación de sklearn para que lo busquéis:

[K-Means on sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

Con esto solo hay que ahora generar los modelos de kmeans, evaluar y pintar la gráfica para los valores de ``k`` que establezcais.




In [ ]:
from sklearn.cluster import KMeans

scores = []
k_values = # definir un rango
for a in k_values:

    # Definir Kmeans y ajustar
    # Guardar la predicción

sns.lineplot(x=k_values, y=scores)

Con el valor que hayáis obtenido de la gráfica, podéis obtener una buena aproximación de Kmeans y con ello podemos pasar a explorar cómo de bien han separado la información los distintos clusters. Para ello, se va a hacer un ``catplot``, seaborn os lo hará solito. Con esto lo que se pretende ver es la distribución de la varaible a predecir en función del cluster que haya determinado Kmeans.

In [ ]:
# Aprender Kmeans con el valor de K obtenido.

kmeans = # Definir y entrenar Kmeans.

# Preparar el catplot.


# Pintar.
ax = sns.catplot(col=, x=, data=, kind='count',col_wrap=4)

Vamos a ver qué tal queda esto pintado. Para ello, repetimos el scatterplot de antes pero usando como color el cluster asignado por kmeans.

In [ ]:
# Entrenar PCA para representar.

# Usar un color por cada cluster.


¿Es bastante parecido no? No es tan bueno como el Random Forest, pero ha conseguido identificar bastante bien los distintos puntos del dataset sin utilizar las etiquetas. De hecho, el diagrama de factor que hemos visto antes muestra que solo un par de clusters son imprecisos. Si no hubieramos tenido etiquetas esta aproximacion nos hubiera ayudado mucho a clasificar los distintos tipos de hongos.